## Inserting Data into Partitions

Let us understand how to use insert to get data into static partitions in Spark Metastore from existing table called as orders.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Managing Tables - DML and Partitioning").
    master("yarn").
    getOrCreate

username = itv002461
spark = org.apache.spark.sql.SparkSession@6ce8365c


org.apache.spark.sql.SparkSession@6ce8365c

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* Let us recap what is covered so far related to partitioned tables.
  * We have created a table called as orders_part with order_month of type INT as partitioned column.
  * We have added 4 static partitions for 201307, 201308, 201309 and 201310 using ALTER TABLE command.
  * Once the table is created and partitions are added we have pre-processed the data to get data into the partitions using LOAD command.
* It is not practical to use LOAD command always. We typically use `INSERT` via stage table to copy data into partitioned table.
* We can pre-create partitions in partitioned tables and insert data into partitions using appropriate `INSERT `command. One need to ensure that required filter condition is applied to get the data relevant to the partition that is being populated.
* We can also create partitions dynamically which we will see as part of the next topic.

In [3]:
%%sql

USE itv002461_retail

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql

ALTER TABLE orders_part ADD PARTITION (order_month=201311)

++
||
++
++



In [5]:
%%sql

SELECT count(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   18389|
+--------+



In [6]:
%%sql

INSERT INTO TABLE orders_part PARTITION (order_month=201311)
  SELECT * FROM orders WHERE order_date LIKE '2013-11%'

++
||
++
++



In [7]:
%%sql

SELECT count(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   24770|
+--------+



In [8]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part" !

drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 01:13 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201307
-rwxr-xr-x   3 itv002461 supergroup      64737 2022-05-30 01:13 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201307/orders_201307
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201308
-rwxr-xr-x   3 itv002461 supergroup     243190 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201308/orders_201308
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201309
-rwxr-xr-x   3 itv002461 supergroup     251262 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201309/orders_201309
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/or

0